# GenParse: Lark Interface

In [1]:
import lark

from genparse.util import LarkStuff
from arsenal import Integerizer
from arsenal.maths import compare
from collections import Counter
from genparse.util import regex_to_greenery, greenery_to_fsa

## Using lark as a front end

In [2]:
grammar2 = r"""
WS: /[ \t\f\r\n]/
STAR: "*"
NUMBER: /\d+/

start: WS "SELECT" WS select_expr WS "FROM" WS from_expr
  [WS "WHERE" WS bool_condition] [WS "GROUP BY" WS var_list] [WS "ORDER BY" WS orderby_expr] WS EOS
EOS: "</s>"
select_expr: STAR | select_list
bool_condition: bool_expr | "(" bool_condition WS "AND" WS bool_condition ")" | "(" bool_condition WS "OR" WS bool_condition ")" 
bool_expr: var "=" value | var ">" value | var "<" value
from_expr: "data"
orderby_expr: var_list WS "ASC" | var_list WS "DESC"
select_list: select_var ("," WS select_var)*
var_list: var ("," WS var)*
select_var: var | "AVG(" var ")" | "MEDIAN(" var ")" | "COUNT(" var ")"
var: "age" | "gender" | "year" | "state_color" | "zipcode" | "vote" | "race_ethnicity"
value: NUMBER | "red" | "blue" | "white" | "black" | "latino" | "republican" | "democrat" | "male" | "female"

"""

In [3]:
grammar1 = """
start: query_expr EOS

EOS: "</s>"

query_expr: select [ "ORDER" "BY" (order_by_expr ",")*  order_by_expr] [ "LIMIT" integer_ ] 

select: "SELECT" [(select_expr ",")*] select_expr "FROM" "data" [ "WHERE" bool_expression ] [ "GROUP" "BY" [(expression ",")*] expression ]

select_expr.0: expression_math [ [ "AS" ] alias ] -> select_expression

?expression_math: expression_product
               | expression_math PLUS expression_product -> expression_add
               | expression_math "-" expression_product -> expression_sub
               | AGGREGATION expression_math /\)/ -> sql_aggregation

?expression: (name | STAR) -> column_name
            | literal

?expression_product: expression_parens
                  | expression_product STAR expression_parens
                  | expression_product "/" expression_parens 

?expression_parens: expression
                  | /\(/ expression_parens STAR expression /\)/ 
                  | /\(/  expression_parens "/" expression /\)/ 
                  | /\(/  expression_parens PLUS expression /\)/
                  | /\(/  expression_parens "-" expression /\)/

bool_expression: bool_parentheses
                 | bool_expression "AND" bool_parentheses 
                 | bool_expression "OR" bool_parentheses
bool_parentheses: comparison_type
                 | /\(/   bool_expression "AND" comparison_type /\)/
                 | /\(/  bool_expression "OR" comparison_type /\)/
comparison_type: equals | not_equals | greater_than | less_than | greater_than_or_equal
| less_than_or_equal | is_null | is_not_null
equals: expression_math "=" expression_math
not_equals: expression_math ("<>" | "!=") expression_math
greater_than: expression_math ">" expression_math
less_than: expression_math "<" expression_math
greater_than_or_equal: expression_math ">=" expression_math
less_than_or_equal: expression_math "<=" expression_math
is_null: expression_math "is" "null"
is_not_null: expression_math "is" "not" "null"

alias: /[A-Za-z]+/
name: /[A-Za-z]+/
PLUS: /\+/

order_by_expr: expression_math ["ASC"] -> order_asc
        | expression_math "DESC" -> order_desc

AGGREGATION.8: ("sum(" | "avg(" | "min(" | "max(" | "count(" "distinct" | "count(")
STAR: /\*/
integer_: /[1-9][0-9]*/
?literal: boolean -> bool
       | integer_ -> number
       | ESCAPED_STRING -> string

boolean: "true" -> true
       | "false" -> false

%import common.WS
%ignore WS
%import common.ESCAPED_STRING
    
"""

The following code is adapted from partenon.

In [4]:
raw_grammar = grammar1

lark_stuff = LarkStuff(raw_grammar)

In [5]:
intern = Integerizer()   # rename nonterminals to integers
g = lark_stuff.convert()
g = g.rename(intern)
assert g.in_cnf()    # lark returns a grammar in CNF
#g = g.cnf

In [6]:
g

1.0: 1 → 2 3
0.08333333333333333: 4 → 5 6
0.058823529411764705: 7 → 5 6
1.0: 8 → 9 10
1.0: 11 → 12 13
1.0: 14 → 15 16
0.09090909090909091: 12 → 5 6
0.08333333333333333: 4 → 4 17
1.0: 18 → 19 20
0.058823529411764705: 7 → 4 17
1.0: 21 → 15 16
1.0: 22 → 19 11
1.0: 23 → OR
1.0: 24 → 25 26
0.09090909090909091: 12 → 4 17
0.25: 27 → ESCAPED_STRING
1.0: 28 → 29 30
1.0: 31 → 23 32
0.058823529411764705: 33 → 34 35
0.1: 36 → 5 37
1.0: 9 → ORDER
1.0: 38 → 39 40
0.06666666666666667: 41 → ESCAPED_STRING
1.0: 42 → 43 44
0.08333333333333333: 4 → 4 45
0.058823529411764705: 7 → 4 45
0.06666666666666667: 41 → 5 46
1.0: 47 → FROM
1.0: 48 → 19 49
1.0: 50 → 47 51
0.09090909090909091: 12 → 4 45
1.0: 52 → 7 21
1.0: 53 → 41 54
1.0: 55 → 19 56
0.1111111111111111: 57 → 41 24
0.1: 36 → __ANON_6
1.0: 58 → GROUP
0.08333333333333333: 4 → TRUE
0.058823529411764705: 7 → TRUE
0.07692307692307693: 59 → 41 54
1.0: 60 → 23 61
1.0: 62 → 2 63
0.09090909090909091: 61 → 5 64
0.09090909090909091: 61 → 5 65
0.07692307692307693: 59 → 5 64
1.0: 66 → __ANON_4
0.07692307692307693: 59 → 5 65
1.0: 67 → 41 68
0.1: 36 → 5 6
0.09090909090909091: 61 → 41 69
1.0: 29 → PLUS
1.0: 70 → 71 72
0.07692307692307693: 59 → 41 69
0.08333333333333333: 4 → STAR
0.058823529411764705: 7 → STAR
1.0: 73 → 43 44
0.058823529411764705: 33 → 34 18
0.06666666666666667: 41 → 5 37
1.0: 74 → 75 76
1.0: 77 → 39 78
1.0: 6 → 36 79
1.0: 80 → 44 81
0.058823529411764705: 7 → 41 82
0.16666666666666666: 44 → FALSE
0.08333333333333333: 83 → 34 84
0.06666666666666667: 41 → __ANON_6
0.09090909090909091: 12 → 41 82
1.0: 85 → 47 2
1.0: 86 → 47 2
1.0: 87 → 7 14
1.0: 88 → 43 89
1.0: 90 → 12 91
0.1: 36 → TRUE
0.5: 92 → 44 81
1.0: 93 → 43 94
0.5: 19 → 12 81
1.0: 81 → COMMA
0.09090909090909091: 61 → 41 95
1.0: 25 → IS
0.06666666666666667: 41 → 5 6
0.07692307692307693: 59 → 41 95
1.0: 96 → 2 77
1.0: 97 → 36 98
1.0: 91 → 47 99
0.5: 100 → 41 101
0.06666666666666667: 41 → 4 17
1.0: 72 → 44 102
1.0: 103 → 2 104
1.0: 46 → 36 70
1.0: 105 → 12 86
0.1: 36 → STAR
1.0: 43 → BY
1.0: 106 → 39 107
1.0: 79 → 108 109
0.058823529411764705: 33 → 34 110
0.06666666666666667: 41 → 4 45
0.1111111111111111: 57 → 41 69
1.0: 111 → __ANON_6
0.08333333333333333: 4 → FALSE
0.058823529411764705: 7 → FALSE
0.09090909090909091: 61 → 41 112
1.0: 113 → 9 114
0.08333333333333333: 83 → 34 35
0.07692307692307693: 59 → 41 112
0.25: 27 → TRUE
1.0: 115 → 41 102
1.0: 63 → 39 59
1.0: 15 → LIMIT
1.0: 16 → __ANON_7
0.5: 116 → TRUE
1.0: 99 → 2 106
0.058823529411764705: 33 → 83 117
1.0: 30 → 44 102
1.0: 118 → 119 120
1.0: 121 → __ANON_5
0.06666666666666667: 41 → TRUE
0.07692307692307693: 59 → 59 122
0.5: 19 → 19 123
1.0: 124 → 43 125
0.07692307692307693: 59 → 41 126
0.5: 127 → 127 128
0.16666666666666666: 44 → STAR
1.0: 68 → 25 129
1.0: 130 → 43 131
1.0: 132 → 19 133
1.0: 71 → STAR
0.058823529411764705: 33 → 34 134
1.0: 135 → 47 1
0.1111111111111111: 57 → 41 95
0.09090909090909091: 61 → 41 54
0.058823529411764705: 7 → 5 97
1.0: 84 → 19 136
0.058823529411764705: 33 → 34 22
0.08333333333333333: 83 → 34 18
1.0: 110 → 12 137
1.0: 138 → 29 4
1.0: 82 → 139 4
0.07692307692307693: 59 → 41 68
1.0: 140 → 39 59
1.0: 107 → 59 141
1.0: 5 → __ANON_1
1.0: 45 → 108 36
1.0: 123 → 12 81
0.09090909090909091: 61 → 41 142
0.06666666666666667: 41 → 41 82
1.0: 143 → 41 142
1.0: 76 → __ANON_6
0.1: 36 → FALSE
0.07692307692307693: 59 → 41 142
1.0: 34 → SELECT
1.0: 144 → 43 44
1.0: 75 → AS
0.058823529411764705: 7 → 41 138
1.0: 128 → 7 81
0.058823529411764705: 33 → 34 55
0.09090909090909091: 12 → 41 138
1.0: 141 → 58 42
1.0: 101 → 145 41
1.0: 51 → 2 146
1.0: 147 → 127 52
1.0: 117 → 9 148
1.0: 149 → EOS
1.0: 112 → 121 41
1.0: 150 → 58 151
0.08333333333333333: 83 → 34 132
1.0: 148 → 43 7
1.0: 152 → 41 69
0.058823529411764705: 7 → 153 115
1.0: 154 → 2 155
0.09090909090909091: 61 → 41 101
0.07692307692307693: 59 → 41 101
0.09090909090909091: 12 → 153 115
1.0: 32 → 57 102
1.0: 89 → 92 44
0.25: 27 → FALSE
1.0: 156 → 2 157
0.5: 116 → FALSE
0.1: 36 → 5 97
0.06666666666666667: 41 → FALSE
1.0: 158 → 47

In [7]:
len(g.rules), len(g.V), len(g.N)

(314, 38, 184)

In [8]:
sorted(g.cnf.V)

['AGGREGATION',
 'AND',
 'AS',
 'ASC',
 'BY',
 'COMMA',
 'DATA',
 'DESC',
 'EOS',
 'EQUAL',
 'ESCAPED_STRING',
 'FALSE',
 'FROM',
 'GROUP',
 'IS',
 'LESSTHAN',
 'LIMIT',
 'MINUS',
 'MORETHAN',
 'NOT',
 'NULL',
 'OR',
 'ORDER',
 'PLUS',
 'SELECT',
 'SLASH',
 'STAR',
 'TRUE',
 'WHERE',
 'WS',
 '__ANON_0',
 '__ANON_1',
 '__ANON_2',
 '__ANON_3',
 '__ANON_4',
 '__ANON_5',
 '__ANON_6',
 '__ANON_7']

In [9]:
#from newton.linking import LinkAnalysis
#f = Integerizer()
#links = LinkAnalysis(g.rename(f))
#links.dfs

In [10]:
#g.language(6)

## Tokenization

We can extract lark's tokenizer in a format that we can build on.  We will even make a DIY tokenizer based on Python's `re` library.

| Terminology  |         |
|--------------|---------|
| tokenization | lexing  |
| tokenizers   | lexers  |
| tokens       | lexemes |



In [11]:
sorted(lark_stuff.terminals, key=lambda t: -t.priority)

[TerminalDef('AGGREGATION', '(?:count\\(distinct|count\\(|sum\\(|avg\\(|min\\(|max\\()'),
 TerminalDef('ESCAPED_STRING', '".*?(?<!\\\\)(\\\\\\\\)*?"'),
 TerminalDef('WS', '(?:[ \t\x0c\r\n])+'),
 TerminalDef('EOS', '</s>'),
 TerminalDef('PLUS', '\\+'),
 TerminalDef('STAR', '\\*'),
 TerminalDef('COMMA', ','),
 TerminalDef('ORDER', 'ORDER'),
 TerminalDef('BY', 'BY'),
 TerminalDef('LIMIT', 'LIMIT'),
 TerminalDef('WHERE', 'WHERE'),
 TerminalDef('GROUP', 'GROUP'),
 TerminalDef('SELECT', 'SELECT'),
 TerminalDef('FROM', 'FROM'),
 TerminalDef('DATA', 'data'),
 TerminalDef('AS', 'AS'),
 TerminalDef('MINUS', '\\-'),
 TerminalDef('__ANON_0', '\\)'),
 TerminalDef('SLASH', '/'),
 TerminalDef('__ANON_1', '\\('),
 TerminalDef('AND', 'AND'),
 TerminalDef('OR', 'OR'),
 TerminalDef('EQUAL', '='),
 TerminalDef('__ANON_2', '<>'),
 TerminalDef('__ANON_3', '!='),
 TerminalDef('MORETHAN', '>'),
 TerminalDef('LESSTHAN', '<'),
 TerminalDef('__ANON_4', '>='),
 TerminalDef('__ANON_5', '<='),
 TerminalDef('IS', 'i

### DIY tokenizer

In [12]:
text = "12 + 24 - 36 * 48 / 60 SELECT table.name AS thing WHERE table.potato IS NOT 'banana'"

for x, y in lark_stuff.simple_tokenizer(text):
    print(f'{x:15s} -> {y!r}')

__ANON_7        -> '12'
PLUS            -> '+'
__ANON_7        -> '24'
MINUS           -> '-'
__ANON_7        -> '36'
STAR            -> '*'
__ANON_7        -> '48'
SLASH           -> '/'
__ANON_7        -> '60'
SELECT          -> 'SELECT'
__ANON_6        -> 'table'
__ANON_6        -> 'name'
AS              -> 'AS'
__ANON_6        -> 'thing'
WHERE           -> 'WHERE'
__ANON_6        -> 'table'
__ANON_6        -> 'potato'
__ANON_6        -> 'IS'
__ANON_6        -> 'NOT'
__ANON_6        -> 'banana'


### Parsing tokenized input

In [13]:
text = 'SELECT name FROM data </s>'

In [14]:
tokens = list(lark_stuff.lex(text))
tokens

[Token('SELECT', 'SELECT'),
 Token('__ANON_6', 'name'),
 Token('FROM', 'FROM'),
 Token('DATA', 'data'),
 Token('EOS', '</s>')]

Call the lark parser on the text:

In [15]:
lark_stuff.instance.parse(text)

ParseError: Parsing failed.

We can call the lark parser on these tokens:

In [ ]:
lark_stuff.parser.parse(tokens, 'start')

We can call our parser on this text to get its total weight

In [ ]:
g([t.type for t in tokens])

### Tokenizer State Machines

**Note**: Tokenizers are FSTs, not FSAs.  However, these libraries implement the kind of restricted FSTs with a separate FSA per token type.

In [18]:
i = 4
token_class = lark_stuff.terminals[i]
m = greenery_to_fsa(regex_to_greenery(token_class.pattern.to_regexp()))
display(token_class)
display(m.min())

TerminalDef('AGGREGATION', '(?:count\\(distinct|count\\(|sum\\(|avg\\(|min\\(|max\\()')

c⋅o⋅u⋅n⋅t⋅(⋅(  + 
 + 
 + 
 + 
 + 	 + (  + 
 + 
 + 
 + 
 + 	)⋅(  + 
 + 
 + 
 + 
 + 	)*⋅(  + 
 + 
 + 
 + 
 + 	) + d⋅i⋅s⋅t⋅i⋅n⋅c⋅(t + t⋅(  + 
 + 
 + 
 + 
 + 	)*⋅(  + 
 + 
 + 
 + 
 + 	))) + (a⋅v⋅g + s⋅u⋅m + m⋅(i⋅n + a⋅x))⋅(( + (⋅(  + 
 + 
 + 
 + 
 + 	)*⋅(  + 
 + 
 + 
 + 
 + 	)) + c⋅o⋅u⋅n⋅t⋅(

In [19]:
#for e in m.arcs(): print(e)

## Prefix Grammar

In [20]:
len(g.cnf.rules), len(g.cnf.prefix_grammar.trim().rules), len(g.cnf.prefix_grammar.trim().rules)/len(g.cnf.rules)

(298, 1202, 4.033557046979865)

In [21]:
#N = G.nullaryremove()    # could be faster with SCC-based prioritization

In [22]:
g.cnf.prefix_grammar.trim().cnf

0.9999999985999923: #0 → 
1.0: 1⚡ → 2 3⚡
0.023776223745427733: 134⚡ → 5 6⚡
0.015384615373160397: 32⚡ → 5 6⚡
0.015384615376611371: 87⚡ → 5 6⚡
0.012587412570082404: 64⚡ → 5 6⚡
0.02377622374538448: 35⚡ → 5 6⚡
0.015384615376611371: 52⚡ → 5 6⚡
0.015384615376611371: 115⚡ → 5 6⚡
0.09999999999194917: 4⚡ → 5 6⚡
0.02377622374299676: 55⚡ → 5 6⚡
0.012587412576513269: 40⚡ → 5 6⚡
0.023776223763853938: 123⚡ → 5 6⚡
0.012587412576536172: 59⚡ → 5 6⚡
0.015384615373160397: 120⚡ → 5 6⚡
0.012587412576513269: 78⚡ → 5 6⚡
0.02377622376385393: 167⚡ → 5 6⚡
0.023776223724570555: 84⚡ → 5 6⚡
0.023776223724570555: 18⚡ → 5 6⚡
0.015384615376611371: 7⚡ → 5 6⚡
0.015384615376611371: 128⚡ → 5 6⚡
0.015384615376611371: 41⚡ → 5 6⚡
0.023776223745427733: 20⚡ → 5 6⚡
0.023776223742996754: 48⚡ → 5 6⚡
0.023776223745427733: 136⚡ → 5 6⚡
0.023776223763853938: 105⚡ → 5 6⚡
0.015384615367878801: 147⚡ → 5 6⚡
0.012587412576536167: 107⚡ → 5 6⚡
0.01258741257804033: 61⚡ → 5 6⚡
0.02377622376381068: 110⚡ → 5 6⚡
0.023776223742953503: 132⚡ → 5 6⚡
0.023776223763853938: 56⚡ → 5 6⚡
0.012587412576536167: 179⚡ → 5 6⚡
0.02377622376381068: 133⚡ → 5 6⚡
0.023776223724527302: 22⚡ → 5 6⚡
0.012587412570082404: 65⚡ → 5 6⚡
0.015384615367878801: 131⚡ → 5 6⚡
0.023776223745427733: 90⚡ → 5 6⚡
0.02377622376385393: 49⚡ → 5 6⚡
0.02377622374538448: 11⚡ → 5 6⚡
0.058823529411764705: 87⚡ → 5 6⚡
0.058823529411764705: 52⚡ → 5 6⚡
0.058823529411764705: 7⚡ → 5 6⚡
0.058823529411764705: 128⚡ → 5 6⚡
0.05882352937837547: 147⚡ → 5 6⚡
0.05882352937837547: 131⚡ → 5 6⚡
1.0: 8⚡ → 9 10⚡
1.0: 11⚡ → 12 13⚡
1.0: 14⚡ → 15 16⚡
0.09090909083863778: 134⚡ → 5 6⚡
0.09090909083847241: 35⚡ → 5 6⚡
0.09090909082934288: 55⚡ → 5 6⚡
0.09090909090909091: 123⚡ → 5 6⚡
0.0909090909090909: 167⚡ → 5 6⚡
0.09090909075888975: 84⚡ → 5 6⚡
0.09090909075888974: 18⚡ → 5 6⚡
0.09090909083863777: 20⚡ → 5 6⚡
0.09090909082934287: 48⚡ → 5 6⚡
0.09090909083863778: 136⚡ → 5 6⚡
0.09090909090909091: 105⚡ → 5 6⚡
0.09090909090892553: 110⚡ → 5 6⚡
0.0909090908291775: 132⚡ → 5 6⚡
0.09090909090909091: 56⚡ → 5 6⚡
0.09090909090892553: 133⚡ → 5 6⚡
0.09090909075872439: 22⚡ → 5 6⚡
0.09090909083863777: 90⚡ → 5 6⚡
0.0909090909090909: 49⚡ → 5 6⚡
0.09090909083847241: 11⚡ → 5 6⚡
0.023776223745427733: 134⚡ → 4 17⚡
0.015384615373160397: 32⚡ → 4 17⚡
0.015384615376611371: 87⚡ → 4 17⚡
0.012587412570082404: 64⚡ → 4 17⚡
0.02377622374538448: 35⚡ → 4 17⚡
0.015384615376611371: 52⚡ → 4 17⚡
0.015384615376611371: 115⚡ → 4 17⚡
0.09999999999194917: 4⚡ → 4 17⚡
0.02377622374299676: 55⚡ → 4 17⚡
0.012587412576513269: 40⚡ → 4 17⚡
0.023776223763853938: 123⚡ → 4 17⚡
0.012587412576536172: 59⚡ → 4 17⚡
0.015384615373160397: 120⚡ → 4 17⚡
0.012587412576513269: 78⚡ → 4 17⚡
0.02377622376385393: 167⚡ → 4 17⚡
0.023776223724570555: 84⚡ → 4 17⚡
0.023776223724570555: 18⚡ → 4 17⚡
0.015384615376611371: 7⚡ → 4 17⚡
0.015384615376611371: 128⚡ → 4 17⚡
0.015384615376611371: 41⚡ → 4 17⚡
0.023776223745427733: 20⚡ → 4 17⚡
0.023776223742996754: 48⚡ → 4 17⚡
0.023776223745427733: 136⚡ → 4 17⚡
0.023776223763853938: 105⚡ → 4 17⚡
0.015384615367878801: 147⚡ → 4 17⚡
0.012587412576536167: 107⚡ → 4 17⚡
0.01258741257804033: 61⚡ → 4 17⚡
0.02377622376381068: 110⚡ → 4 17⚡
0.023776223742953503: 132⚡ → 4 17⚡
0.023776223763853938: 56⚡ → 4 17⚡
0.012587412576536167: 179⚡ → 4 17⚡
0.02377622376381068: 133⚡ → 4 17⚡
0.023776223724527302: 22⚡ → 4 17⚡
0.012587412570082404: 65⚡ → 4 17⚡
0.015384615367878801: 131⚡ → 4 17⚡
0.023776223745427733: 90⚡ → 4 17⚡
0.02377622376385393: 49⚡ → 4 17⚡
0.02377622374538448: 11⚡ → 4 17⚡
1.0: 18⚡ → 19 20⚡
0.058823529411764705: 87⚡ → 4 17⚡
0.058823529411764705: 52⚡ → 4 17⚡
0.058823529411764705: 7⚡ → 4 17⚡
0.058823529411764705: 128⚡ → 4 17⚡
0.05882352937837547: 147⚡ → 4 17⚡
0.05882352937837547: 131⚡ → 4 17⚡
1.0: 21⚡ → 15 16⚡
1.0: 22⚡ → 19 11⚡
1.0: 24⚡ → 25 26⚡
0.09090909083863778: 134⚡ → 4 17⚡
0.09090909083847241: 35⚡ → 4 17⚡
0.09090909082934288: 55⚡ → 4 17⚡
0.09090909090909091: 123⚡ → 4 17⚡
0.0909090909090909: 167⚡ → 4 17⚡
0.09090909075888975: 84⚡ → 4 17⚡
0.09090909075888974: 18⚡ → 4 17⚡
0.09090909083863777: 20⚡ → 4 17⚡


In [23]:
g.prefix_weight([t.type for t in tokens])

0.0